In [ ]:
import pandas as pd

# Carica il dataset
df = pd.read_excel("../data/raw/final_data_mondinoxicot.xlsx")  # Adatta path se necessario

# Calcola % NaN per colonna
nan_percent = df.isna().mean().sort_values(ascending=False) * 100
nan_df = nan_percent[nan_percent > 0].to_frame(name="Percentuale NaN")
nan_df.head(20)  # Mostra le 20 peggiori

In [ ]:
# 🔹 Variabili chiave da tracciare per missingness
vars_to_flag = [
    "Updrs-III", "Duration (years)", "AgeOnset", "OnsetCategory",
    "DoubleSupport", "SingleSupport", "GaitSpeed", "StrideLength"
]

# 🔹 Crea colonne binarie che indicano se il valore era mancante
for col in vars_to_flag:
    if col in df.columns:
        safe_colname = col.replace(" ", "_").replace("-", "_")
        df[f"{safe_colname}_was_missing"] = df[col].isna().astype(int)

# 🔹 Controllo rapido
df[[c for c in df.columns if "_was_missing" in c]].head()

In [ ]:
# Colonne flag create
flag_cols = [c for c in df.columns if c.endswith("_was_missing")]

# Quanti flag per soggetto
df["missing_flags_sum"] = df[flag_cols].sum(axis=1)

# Distribuzione globale
print("Distribuzione numero di flag per soggetto:")
print(df["missing_flags_sum"].value_counts().sort_index())

# Distribuzione per classe (se target_bin presente)
if "target_bin" in df.columns:
    print("\nDistribuzione flag per classe:")
    print(df.groupby("target_bin")["missing_flags_sum"].describe().round(2))

# Quanti hanno almeno 1 flag
n_any = (df["missing_flags_sum"] > 0).sum()
print(f"\nSoggetti con ≥1 flag: {n_any} su {len(df)} "
      f"({n_any/len(df)*100:.1f}%)")

# (Facoltativo) Esporta un report veloce
df[["missing_flags_sum"] + flag_cols].to_csv("../tables/missing_flags_report.csv", index=False)

In [ ]:
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

# Colonne numeriche (escludi i flag *_was_missing)
num_cols_all = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols_all if not c.endswith("_was_missing")]

# Colonne categoriche (object)
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()

# Imputazione numeriche -> mediana (robusta)
if len(num_cols) > 0:
    imputer_num = SimpleImputer(strategy="median")
    df[num_cols] = imputer_num.fit_transform(df[num_cols])

# Imputazione categoriche -> moda
if len(cat_cols) > 0:
    imputer_cat = SimpleImputer(strategy="most_frequent")
    df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])

# (Facoltativo: controllo rapido)
print(f"Imputate numeriche: {len(num_cols)} | categoriche: {len(cat_cols)}")

In [ ]:
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

# 1) Ordinal encoding per l'esordio (early < middle < late)
if "OnsetCategory" in df.columns:
    oe = OrdinalEncoder(categories=[['1', '2', '3']])  # se già stringhe
    # Coerciamo a stringhe per sicurezza
    df["OnsetCategory"] = df["OnsetCategory"].astype(str)
    df[["OnsetCategory"]] = oe.fit_transform(df[["OnsetCategory"]])

# 2) One-hot per categoriche a bassa cardinalità (≤4 livelli)
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
low_card_cat = [col for col in cat_cols if df[col].nunique() <= 4]

if len(low_card_cat) > 0:
    df = pd.get_dummies(df, columns=low_card_cat, drop_first=True)

# 3) Drop variabili identificative e potenziale leakage
cols_to_drop = ["ID", "Surname", "Name", "ProdromalCount", "Center"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors="ignore")

# Se dopo i dummies fosse comparso qualcosa tipo "Center_<sito>", rimuovilo
center_dummy_cols = [c for c in df.columns if c.startswith("Center_")]
if center_dummy_cols:
    df = df.drop(columns=center_dummy_cols, errors="ignore")

# 4) Controlli
print("✅ Shape finale:", df.shape)
print("🧼 Tipi di dato post-cleaning:\n", df.dtypes.value_counts())
print("🔍 Colonne non numeriche residue:", df.select_dtypes(exclude=[np.number, 'bool']).columns.tolist())

In [ ]:
import pandas as pd
import numpy as np

# --- Funzione robusta per coercizione numerica ---
def to_numeric_safe(s: pd.Series):
    s = s.astype(str).str.replace(r"\s+(?=\d)", "", regex=True)  # es. "6 2" -> "62"
    s = s.str.replace(",", ".", regex=False)                     # virgola decimale -> punto
    return pd.to_numeric(s, errors="coerce")

# --- 1) Evaluation Date -> datetime, poi rimuoviamo per evitare leakage ---
if "Evaluation Date" in df.columns:
    df["Evaluation_Date"] = pd.to_datetime(df["Evaluation Date"], errors="coerce")
    df = df.drop(columns=["Evaluation Date"], errors="ignore")
# rimuovi anche la versione datetime dai dati per i modelli
df = df.drop(columns=["Evaluation_Date"], errors="ignore")

# --- 2) Age Onset -> numerico + rename coerente ad 'AgeOnset' ---
if "Age Onset" in df.columns:
    df["AgeOnset"] = to_numeric_safe(df["Age Onset"])
    df = df.drop(columns=["Age Onset"], errors="ignore")

# --- 3) LEDD e Updrs-III -> numerici robusti ---
for col in ["LEDD", "Updrs-III"]:
    if col in df.columns:
        df[col] = to_numeric_safe(df[col])

# --- Controllo finale ---
print("🧼 Tipi di dato attuali:\n", df.dtypes.value_counts())
non_num = df.select_dtypes(exclude=[np.number, 'bool', 'datetime64[ns]']).columns.tolist()
print("🔍 Colonne non numeriche residue:", non_num if len(non_num) > 0 else "Nessuna")

In [ ]:
print("📐 Shape finale del dataset:", df.shape)

# 1. Missing values
print("\n🔎 Valori NaN residui:")
nan_residui = df.isna().sum()[df.isna().sum() > 0]
if nan_residui.empty:
    print("✅ Nessun NaN residuo")
else:
    print(nan_residui)

# 2. Tipi di dato
print("\n📊 Tipi di dato:")
print(df.dtypes.value_counts())

# 3. Colonne non numeriche
non_numeric_cols = df.select_dtypes(exclude=["int64", "float64"]).columns
if len(non_numeric_cols) > 0:
    print("\n⚠️ Colonne non numeriche:", non_numeric_cols.tolist())
else:
    print("\n✅ Tutte le colonne sono numeriche")

# 4. Colonne con varianza nulla
from sklearn.feature_selection import VarianceThreshold
constant_filter = VarianceThreshold(threshold=0)
constant_filter.fit(df)
zero_var_cols = df.columns[~constant_filter.get_support()]
if len(zero_var_cols) > 0:
    print("\n⚠️ Colonne con varianza nulla:", zero_var_cols.tolist())
else:
    print("\n✅ Nessuna colonna con varianza nulla")

# 5. Statistiche descrittive (prime 10 righe, resto su CSV)
desc = df.describe().T
print("\n📈 Statistiche di base (prime 10 variabili):")
display(desc.head(10))
desc.to_csv("../tables/descriptive_postclean.csv")

# 6. Outlier check (boxplot ordinato per varianza, escluso LEDD)
import matplotlib.pyplot as plt
import seaborn as sns

cols_to_plot = [
    col for col in df.columns
    if df[col].nunique() > 10 and df[col].dtype in ['float64', 'int64'] and col != 'LEDD'
]

df_melted = pd.melt(df[cols_to_plot])
order = df[cols_to_plot].var().sort_values(ascending=False).index

plt.figure(figsize=(15, 8))
sns.boxplot(x='variable', y='value', data=df_melted, order=order)
plt.xticks(rotation=90)
plt.title("📦 Boxplot delle feature continue (escluso LEDD)")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew

# STEP 0 — Impostazioni
log_vars = [c for c in ["LEDD", "Duration (years)"] if c in df.columns]  # trasformazione log
ordinals_to_exclude = [c for c in ["OnsetCategory"] if c in df.columns]   # evita ordinali nei test outlier

# STEP 1 — Seleziona feature numeriche continue (no target/binari/ordinali)
cols_to_plot = [
    col for col in df.columns
    if df[col].dtype in ['float64', 'int64']
    and df[col].nunique() > 10
    and col not in ordinals_to_exclude
]

# STEP 2 — Funzione outlier via IQR
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return ((series < lower) | (series > upper)).sum()

# STEP 3 — Report outlier e skewness
outlier_report = {}
skew_report = {}
for col in cols_to_plot:
    outlier_report[col] = detect_outliers_iqr(df[col])
    skew_report[col] = skew(df[col], nan_policy='omit')

df_outlier_skew = pd.DataFrame({
    "n_outliers": pd.Series(outlier_report),
    "skewness": pd.Series(skew_report)
}).sort_values(by="n_outliers", ascending=False)

print("📊 Report outlier e skewness:")
display(df_outlier_skew)

# STEP 4 — Regole:
#   - log1p su LEDD e Duration (years)
#   - winsorize se (outlier > 3) o (|skew| > 1.5), escludendo le variabili loggate
def winsorize_series(series, lower_quantile=0.01, upper_quantile=0.99):
    lower = series.quantile(lower_quantile)
    upper = series.quantile(upper_quantile)
    return series.clip(lower, upper)

# 4a) Log-transform in-place (manteniamo lo stesso nome per semplicità)
for col in log_vars:
    df[col] = np.log1p(df[col])

# 4b) Selezione per winsorizing
candidates = df_outlier_skew[
    (df_outlier_skew["n_outliers"] > 3) | (df_outlier_skew["skewness"].abs() > 1.5)
].index.tolist()

features_to_winsorize = [c for c in candidates if c not in log_vars]

print(f"\n✅ Feature selezionate per Winsorizing ({len(features_to_winsorize)}):")
print(features_to_winsorize)

for col in features_to_winsorize:
    df[col] = winsorize_series(df[col])

# STEP 5 — Verifica visiva (sceglie la prima variabile modificata, se c'è; altrimenti una qualsiasi continua)
if len(features_to_winsorize) > 0:
    esempio = features_to_winsorize[0]
elif len(cols_to_plot) > 0:
    esempio = cols_to_plot[0]
else:
    esempio = None

if esempio is not None:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.boxplot(y=df[esempio], ax=axes[0])
    axes[0].set_title(f"🟥 Dopo Winsorizing/Log — {esempio}")
    sns.histplot(df[esempio], ax=axes[1], kde=True)
    axes[1].set_title(f"📈 Distribuzione dopo trasformazioni — {esempio}")
    plt.tight_layout()
    plt.suptitle("Controllo Qualità Preprocessing Outlier", y=1.02, fontsize=14)
    plt.show()
else:
    print("ℹ️ Nessuna variabile disponibile per il plot di controllo.")

In [ ]:
import pandas as pd
import numpy as np

# 🔹 Escludiamo target e variabili non adatte alla correlazione
exclude_cols = [
    "target_bin", "Hyposmia", "REM", "Depression", "Constipation",
    "ID", "Evaluation_Date", "AgeOnset", "OnsetCategory", 
    "LEDD", "Updrs_III"
]

X_numeriche = df.drop(columns=[col for col in exclude_cols if col in df.columns])

# 🔹 Calcolo della matrice di correlazione Spearman
corr_matrix = X_numeriche.corr(method="spearman")

# 🔹 Lista per salvare le coppie ad alta correlazione
correlated_pairs = []

threshold = 0.6  # più stringente

for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        corr_value = corr_matrix.iloc[i, j]
        if abs(corr_value) > threshold:
            f1 = corr_matrix.columns[i]
            f2 = corr_matrix.columns[j]
            correlated_pairs.append((f1, f2, corr_value))

# 🔹 Ordinamento per forza di correlazione
correlated_df = pd.DataFrame(correlated_pairs, columns=["Feature_1", "Feature_2", "Spearman_r"])
correlated_df["|r|"] = correlated_df["Spearman_r"].abs()
correlated_df = correlated_df.sort_values(by="|r|", ascending=False).drop(columns="|r|")

# 🔹 Output
print(f"🔎 Coppie di feature con Spearman > {threshold}:")
display(correlated_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib_venn import venn3
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, RFE

# --- Palette e label ---
palette = {0: "#003f5c", 1: "#bc5090"}
main_color = palette[1]
alt_color  = palette[0]

# ======================
# STEP 0 — Rimozione feature non utilizzabili / ridondanti (logica aggiornata)
# ======================

# a) Colonne da escludere (identificativi, target, prodromi, cliniche pure)
base_exclude = [
    "target_bin", "ID", "Surname", "Name", "Center", "Evaluation_Date", "ProdromalCount",
    "Constipation", "Hyposmia", "REM", "Depression",
    "Updrs-III", "H-Y"  # scale cliniche (le useremo come covariate nei modelli, non nella FS biomeccanica)
]

# b) Ridondanze note (da analisi di collinearità): tieni Gait Speed, droppa Stride Length; droppa Stance & Swing; tra HRV/HRAP tieni HR V
redundant_cols = [
    "Stance", "Swing", "Gait Speed", "HR AP"
]

# c) Eventuali flag di missingness
flag_cols = [c for c in df.columns if c.endswith("_was_missing") or c == "missing_flags_sum"]

cols_to_drop = [c for c in (base_exclude + redundant_cols + flag_cols) if c in df.columns]

df_filtered = df.drop(columns=cols_to_drop, errors="ignore").copy()

# X e y
y = df["target_bin"].astype(int)
X = df_filtered.copy()

print(f"🔧 FS: partiamo da {X.shape[1]} feature (dopo esclusioni).")

# ======================
# STEP 1 — FILTER: ANOVA F-test
# ======================

filter_selector = SelectKBest(score_func=f_classif, k="all")
filter_selector.fit(X, y)

filter_scores = pd.DataFrame({
    "Feature": X.columns,
    "F_score": filter_selector.scores_,
    "p_value": filter_selector.pvalues_
}).sort_values(by="F_score", ascending=False)

features_filter = filter_scores.query("p_value < 0.05")["Feature"].tolist()

# GRAFICO 1 — Barplot F-score (solo significanti)
plt.figure(figsize=(10, 6))
sig_df = filter_scores.query("p_value < 0.05").copy()
sns.barplot(data=sig_df, x="F_score", y="Feature", color=main_color)
plt.title("FILTER — ANOVA F-test (p < 0.05)", fontsize=14)
plt.xlabel("F-score"); plt.ylabel("")
sns.despine(); plt.tight_layout(); plt.show()

# ======================
# STEP 2 — WRAPPER: RFE con RandomForest
# ======================

# Selezioniamo solo le feature passate dal filtro (se vuote, fallback a tutte)
X_filter = X[features_filter] if len(features_filter) > 0 else X.copy()

rf_clf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
rfe_selector = RFE(estimator=rf_clf, n_features_to_select=min(10, X_filter.shape[1]), step=1)
rfe_selector.fit(X_filter, y)

features_wrapper = X_filter.columns[rfe_selector.support_].tolist()

# GRAFICO 2 — Ranking RFE (1 = selezionata)
rfe_ranking = pd.Series(rfe_selector.ranking_, index=X_filter.columns).sort_values(ascending=True)
plt.figure(figsize=(10, 6))
sns.barplot(x=rfe_ranking.values, y=rfe_ranking.index, palette=[alt_color if r>1 else main_color for r in rfe_ranking.values])
plt.title("WRAPPER — RFE Ranking (1 = Selezionata)", fontsize=14)
plt.xlabel("Ranking"); plt.ylabel("Feature")
sns.despine(); plt.tight_layout(); plt.show()

# ======================
# STEP 3 — EMBEDDED: Feature Importance da RF
# ======================

rf_clf.fit(X_filter[features_wrapper], y)
importances = rf_clf.feature_importances_

df_importance = pd.DataFrame({
    "Feature": features_wrapper,
    "Importance": importances
}).sort_values(by="Importance", ascending=False)

mean_importance = df_importance["Importance"].mean()
features_embedded = df_importance.loc[df_importance["Importance"] > mean_importance, "Feature"].tolist()

# GRAFICO 3 — Feature Importance
plt.figure(figsize=(10, 6))
sns.barplot(data=df_importance, x="Importance", y="Feature", color=alt_color)
plt.title("EMBEDDED — Random Forest Feature Importance", fontsize=14)
plt.xlabel("Importance Score"); plt.ylabel("")
sns.despine(); plt.tight_layout(); plt.show()

# ======================
# STEP 4 — VENN Diagram
# ======================

plt.figure(figsize=(8, 8))
venn3(
    [set(features_filter), set(features_wrapper), set(features_embedded)],
    set_labels=("Filter", "Wrapper", "Embedded"),
    set_colors=(main_color, alt_color, "#ffa600"),
    alpha=0.7
)
plt.title("Convergenza tra Metodi di Feature Selection", fontsize=14)
plt.tight_layout(); plt.show()

# ======================
# STEP 5 — Output finale
# ======================

df_selected = df[features_embedded + ["target_bin"]].copy()

print("✅ FILTER →", len(features_filter), "feature selezionate")
print("✅ WRAPPER →", len(features_wrapper), "feature selezionate")
print("✅ EMBEDDED →", len(features_embedded), "feature finali top")
print("\n🎯 FEATURE SELEZIONATE (da passare a CTGAN / modelli):")
print(features_embedded)

# ======================
# STEP 6 — Tabella completa con valori numerici
# ======================

# 1) F-Score + p-value (Filter)
filter_data = filter_scores.set_index("Feature")[["F_score", "p_value"]]

# 2) RFE ranking (Wrapper)
rfe_data = pd.DataFrame({"RFE_ranking": rfe_selector.ranking_}, index=X_filter.columns)

# 3) Feature Importance (Embedded)
embedded_data = df_importance.set_index("Feature")[["Importance"]]

# Unione e indicatori di selezione
summary_df_complete = pd.concat([filter_data, rfe_data, embedded_data], axis=1)
summary_df_complete["Filter_selected"]   = summary_df_complete.index.isin(features_filter).astype(int)
summary_df_complete["Wrapper_selected"]  = summary_df_complete.index.isin(features_wrapper).astype(int)
summary_df_complete["Embedded_selected"] = summary_df_complete.index.isin(features_embedded).astype(int)
summary_df_complete["Totale_metodi"]     = summary_df_complete[["Filter_selected","Wrapper_selected","Embedded_selected"]].sum(axis=1)

summary_df_complete = summary_df_complete.sort_values(by=["Totale_metodi","Importance","F_score"], ascending=[False, False, False])

print("\n📊 Tabella dettagliata (Filter/Wrapper/Embedded):")
print(summary_df_complete.fillna("").to_string())

# (Opzionale) salvataggio
# summary_df_complete.to_excel("../output/feature_selection/riepilogo_feature_selection_completo.xlsx")
# df_selected.to_csv("../output/feature_selection/selected_features_embedded.csv", index=False)

In [ ]:
import pandas as pd

# 🔹 CORE FEATURES (selezionate da Filter + Wrapper + Embedded)
core_features = [
    "MSE ML", 
    "iHR V", 
    "MSE V", 
    "MSE AP", 
    "Weigth"
]

# 🔹 COVARIATE CLINICHE (per aggiustamento nei modelli)
covariates = [
    "Age",
    "Sex (M=1, F=2)",
    "H-Y",
    "Gait Speed",
    "Duration (years)"
]

# 🔹 Variabili target / prodromi
targets = [
    "target_bin",      # triade vs non-triade
    "Constipation", 
    "Hyposmia", 
    "REM", 
    "Depression"
]

# === 1. Dataset core (solo feature principali + target) ===
df_core = df[core_features + ["target_bin"]].copy()

# === 2. Dataset core + covariates (per modelli supervisionati) ===
df_core_cov = df[core_features + covariates + ["target_bin"]].copy()

# === 3. Dataset completo con prodromi (per analisi descrittive) ===
df_with_prodromi = df[core_features + covariates + targets].copy()

# 🔹 Percorsi di output
csv_core = "../data/processed/dataset_core.csv"
xlsx_core = "../data/processed/dataset_core.xlsx"

csv_cov = "../data/processed/dataset_core_covariates.csv"
xlsx_cov = "../data/processed/dataset_core_covariates.xlsx"

csv_prod = "../data/processed/dataset_with_prodromi.csv"
xlsx_prod = "../data/processed/dataset_with_prodromi.xlsx"

# 🔹 Salvataggi
df_core.to_csv(csv_core, index=False)
df_core.to_excel(xlsx_core, index=False)

df_core_cov.to_csv(csv_cov, index=False)
df_core_cov.to_excel(xlsx_cov, index=False)

df_with_prodromi.to_csv(csv_prod, index=False)
df_with_prodromi.to_excel(xlsx_prod, index=False)

print("✅ Dataset salvati in:")
print(f"- Core: {csv_core} / {xlsx_core}")
print(f"- Core + covariate: {csv_cov} / {xlsx_cov}")
print(f"- Core + covariate + prodromi: {csv_prod} / {xlsx_prod}")